<a href="https://colab.research.google.com/github/kenttw/kent-ai-learning-notebook/blob/master/Machine_Learning_Exercise/CreditCard/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 酨入資料

In [ ]:
# 如果有使用 coloab 再執行此 cell
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
f1id = '1SvJ_IiHr-ndJDG_sBf6NCn0lMKUxPIlf'
f2id = '1lZPv46zul32Xbr1qHES66YRzMa-A7MzB'


!pip3 install gdown
import gdown
url = 'https://drive.google.com/uc?id=%s'%(f1id)
output = 'train.csv'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?id=%s'%(f2id)
output = 'test.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1SvJ_IiHr-ndJDG_sBf6NCn0lMKUxPIlf
To: /content/train.csv
132MB [00:02, 57.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lZPv46zul32Xbr1qHES66YRzMa-A7MzB
To: /content/test.csv
36.2MB [00:00, 81.3MB/s]


'test.csv'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
! mkdir data model submit

# Load Data and Overview

In [ ]:
df = pd.read_csv('train.csv').set_index('txkey')
df.shape

(1521787, 22)

In [ ]:
y_train = df['fraud_ind']
del df['fraud_ind']

In [ ]:
# testing data 的要放進來，這樣 labeling encoding 才不會漏掉
df_test = pd.read_csv('test.csv').set_index('txkey')
df_test.shape

(421665, 21)

In [ ]:
df_test.head(2)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
txkey,,,,,,,,,,,,,,,,,,,,,
592489,6881,163188,116799,513.8,5,0,N,0,N,N,5,N,0,102,215328.0,457,59360,N,0,102,0
592452,6881,163188,116799,513.8,5,0,N,0,N,N,5,N,0,102,222007.0,457,59360,N,0,102,0


In [ ]:
df = pd.concat((df, df_test), 0)
df.shape

(1943452, 21)

In [ ]:
df.keys()

Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt', 'loctm',
       'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd'],
      dtype='object')

In [ ]:
# 欄位類別判定
col_names_cont = [] # 數值型資料
col_names_disc = [] # 類別型資料
col_has_na = [] # 待補NA資料
for c in df.keys():
    uni = df[c].unique()
    n_na = pd.isna(df[c]).sum() # NA 數量
    if n_na > 0:
        col_has_na.append(c)       
    if len(uni) < 200 :
        print(f"{c}: uni={uni}")
        col_names_disc.append(c)
    else:
        info = [ df[c].max(), df[c].min(), df[c].mean(), df[c].std()]
        info = [ round(x,2) for x in info ]
        offset = 1 if n_na else 0
        diversity = (len(uni)-offset)/(len(df)-n_na)
        print(f"{c}: dtype={df[c].dtype}, n_na={n_na}")
        print("       max={}, min={}, mean={}, std={}, diversity={:.2f}%".format(*info, diversity*100 ) )
        if diversity == 1.0:
            print(f"       Delete col [{c}] due to diversity is 100% ")
        else:
            col_names_cont.append(c)

acqic: dtype=int64, n_na=0
       max=6884, min=0, mean=6008.0, std=1502.42, diversity=0.40%
bacno: dtype=int64, n_na=0
       max=163884, min=1, mean=82090.27, std=47362.49, diversity=6.26%
cano: dtype=int64, n_na=0
       max=213334, min=0, mean=108917.04, std=60903.63, diversity=8.50%
conam: dtype=float64, n_na=0
       max=7208.77, min=0.0, mean=654.72, std=402.81, diversity=4.79%
contp: uni=[5 3 6 2 4 0 1]
csmcu: uni=[ 0 62 74 61 13 67 60 26 49 10 71 56 20 38 22  4 47 54 75 32 66 29 28 14
 50 30 31 45 24 40 35 59  3 48 63 52 41 23 17 16 73 51 53 44 18 12 15  5
 70 21 39 27 42 11 72  6 25  7 43  2 19 37  1 64 57 36 68 55  9 46 34 69]
ecfg: uni=['N' 'Y']
etymd: uni=[ 0  2  5  4  8  1  9  6  3  7 10]
flbmk: uni=['N' 'Y' nan]
flg_3dsmk: uni=['N' 'Y' nan]
hcefg: uni=[5 0 3 1 2 7 8 9 6]
insfg: uni=['N' 'Y']
iterm: uni=[0 1 2 8 3 4 7 6 5]
locdt: uni=[33  9  6  5  7 10  8 11 13 14 22 21 25 24 15 12 19 28 18 17 16 23 31 26
  2 30  4 20 27 29  1  3 34 66 32 67 70 69 65 35 39 43 38 45 48 54 

- 看起來數值型資料都沒有缺失值(n_na==0)

In [ ]:
col_has_na

['flbmk', 'flg_3dsmk']

In [ ]:
# 處理缺失值
df = df.fillna("NA")

# Data Preprocessing

## 數值型

In [ ]:
df_cont = df[col_names_cont].copy()
df_cont.head()

,acqic,bacno,cano,conam,loctm,mcc,mchno,scity
txkey,,,,,,,,
516056,6881,113261,38038,513.80,172652.0,457,59333,0
4376,0,134508,45725,465.62,105114.0,451,0,5817
483434,6881,15408,188328,513.80,152458.0,457,59333,0
1407164,6716,157159,29967,1016.11,172946.0,247,50436,3281
1051004,5975,105985,81305,713.66,182129.0,263,93775,5817


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_cont = scaler.fit_transform(df_cont)
X_cont.shape, type(X_cont)

((1521787, 8), numpy.ndarray)

In [ ]:
df_cont = pd.DataFrame(data=X_cont, index=df.index, columns=col_names_cont)
df_cont.head()

,acqic,bacno,cano,conam,loctm,mcc,mchno,scity
txkey,,,,,,,,
516056,0.581061,0.658131,-1.163791,-0.349849,0.505301,2.041755,0.111695,-2.401805
4376,-3.998885,1.106735,-1.037575,-0.469459,-0.790491,1.964800,-1.813266,0.536349
483434,0.581061,-1.407913,1.303879,-0.349849,0.117856,2.041755,0.111695,-2.401805
1407164,0.471238,1.584983,-1.296311,0.897173,0.510941,-0.651666,-0.176953,-0.744579
1051004,-0.021966,0.504508,-0.453373,0.146318,0.687127,-0.446453,1.229109,0.536349


In [ ]:
df_cont.describe()

,acqic,bacno,cano,conam,loctm,mcc,mchno,scity
count,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06,1.521787e+06
mean,-6.344073e-15,-2.975369e-17,2.534324e-16,3.434595e-15,2.775836e-16,-3.063463e-14,-6.876544e-15,-4.602563e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.998885e+00,-1.733213e+00,-1.788351e+00,-1.625396e+00,-2.807219e+00,-3.819643e+00,-1.813266e+00,-2.401805e+00
25%,-1.730721e-02,-8.590612e-01,-8.420198e-01,-5.403618e-01,-6.788851e-01,-6.131887e-01,-7.175244e-01,-4.849587e-01
50%,4.712381e-01,-3.763900e-03,1.425131e-02,-1.559600e-01,9.989811e-02,-4.336273e-01,1.125712e-01,5.363494e-01
75%,5.065145e-01,8.650040e-01,8.695537e-01,3.788859e-01,7.580774e-01,5.796122e-01,7.562472e-01,5.363494e-01
max,5.830577e-01,1.726973e+00,1.714463e+00,1.627091e+01,1.719915e+00,2.067407e+00,1.538359e+00,9.677031e-01


## 類別型

In [ ]:
df_disc = df[col_names_disc].copy()
df_disc.head()

,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd
txkey,,,,,,,,,,,,,
516056,5,0,N,0,N,N,5,N,0,33,N,102,0
4376,5,0,N,2,N,N,0,N,0,9,N,102,0
483434,5,0,N,0,N,N,5,N,0,6,N,102,0
1407164,5,62,N,5,N,N,5,N,0,5,N,102,0
1051004,5,62,N,4,N,N,5,N,0,6,N,102,0


In [ ]:
df_disc.shape

(1521787, 13)

In [ ]:

from sklearn.preprocessing import LabelEncoder
les = {}
for c in col_names_disc:
    le = LabelEncoder()
    df_disc.loc[:,c] = le.fit_transform(df_disc.loc[:,c])
    les.update({c:le})
df_disc.head()

,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd
txkey,,,,,,,,,,,,,
516056,5,0,0,0,0,0,4,0,0,32,0,98,0
4376,5,0,0,2,0,0,0,0,0,8,0,98,0
483434,5,0,0,0,0,0,4,0,0,5,0,98,0
1407164,5,59,0,5,0,0,4,0,0,4,0,98,0
1051004,5,59,0,4,0,0,4,0,0,5,0,98,0


In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
X_disc = ohe.fit_transform(df_disc)
# cut_point = ohe.feature_indices_
# print("feature cut point: ", cut_point)
X_disc.shape, type(X_disc)

((1521787, 318), numpy.ndarray)

In [ ]:
new_col_names_disc = []
for c in col_names_disc: 
    le = les[c]
    new_col_names_disc += [ c+'_'+str(cl) for cl in le.classes_ ]
assert len(new_col_names_disc) == X_disc.shape[1]

In [ ]:
df_disc = pd.DataFrame(data=X_disc, index=df.index, columns=new_col_names_disc)
df_disc.head()

,contp_0,contp_1,contp_2,contp_3,contp_4,contp_5,contp_6,csmcu_0,csmcu_1,csmcu_2,csmcu_3,csmcu_4,csmcu_5,csmcu_6,csmcu_7,csmcu_9,csmcu_10,csmcu_11,csmcu_12,csmcu_13,csmcu_14,csmcu_15,csmcu_16,csmcu_17,csmcu_18,csmcu_19,csmcu_20,csmcu_21,csmcu_22,csmcu_23,csmcu_24,csmcu_25,csmcu_26,csmcu_27,csmcu_28,csmcu_29,csmcu_30,csmcu_31,csmcu_32,csmcu_34,...,stocn_72,stocn_73,stocn_74,stocn_75,stocn_76,stocn_77,stocn_78,stocn_79,stocn_80,stocn_81,stocn_82,stocn_83,stocn_84,stocn_85,stocn_86,stocn_87,stocn_88,stocn_89,stocn_90,stocn_91,stocn_92,stocn_93,stocn_94,stocn_95,stocn_96,stocn_97,stocn_98,stocn_99,stocn_100,stocn_101,stocn_102,stocn_104,stocn_105,stocn_106,stocn_107,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4376,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
483434,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1407164,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1051004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
df_disc.columns

Index(['contp_0', 'contp_1', 'contp_2', 'contp_3', 'contp_4', 'contp_5',
       'contp_6', 'csmcu_0', 'csmcu_1', 'csmcu_2',
       ...
       'stocn_102', 'stocn_104', 'stocn_105', 'stocn_106', 'stocn_107',
       'stscd_0', 'stscd_1', 'stscd_2', 'stscd_3', 'stscd_4'],
      dtype='object', length=318)

In [ ]:
df = pd.concat((df_cont, df_disc), 1)
df.shape

(1943452, 367)

In [ ]:
df.to_csv("./data/df_all.csv")

In [ ]:
y_train.to_csv("df_all_y.csv")

# 轉換 Testing 資料

In [ ]:
df_test

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
txkey,,,,,,,,,,,,,,,,,,,,,
592489,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,102,215328.0,457,59360,N,0,102,0
592452,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,102,222007.0,457,59360,N,0,102,0
590212,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,100,170013.0,457,59360,N,0,102,0
590209,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,100,165914.0,457,59360,N,0,102,0
592488,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,102,215311.0,457,59360,N,0,102,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187507,5975,147326,79511,633.76,5,62,N,2,N,N,5,N,0,116,111228.0,343,87282,N,5817,102,0
1182598,6716,14305,136493,952.84,5,62,N,5,N,N,8,N,0,107,85839.0,245,48784,N,5859,102,0
898724,5975,156543,137963,713.42,5,62,N,4,N,N,5,N,0,111,184921.0,263,98326,N,5817,102,0
